In [ ]:
"""
@author: Kealan Hennessy
@author: Maziar Raissi
"""

import tensorflow as tf
import numpy as np
import scipy.io
import time
from Burgers_custom import PhysicsInformedNN

np.random.seed(1234)
tf.random.set_seed(1234)
tf.keras.backend.set_floatx("float32")

In [ ]:
data_fpath = './burgers_shock.mat'
lambda_1_true, lambda_2_true = 1.0, (0.01/np.pi)

def get_all(fpath):
    data = scipy.io.loadmat(fpath)
    x, t, u_exact = data['x'], data['t'], np.real(data['usol']).T
    xx, tt = np.meshgrid(x, t)
    X_all = np.hstack((xx.flatten()[:,None], tt.flatten()[:,None]))
    y_all = u_exact.flatten()[:,None]
    lb = X_all.min(0) # lower domain bound
    ub = X_all.max(0) # upper domain bound
    return X_all, y_all, lb, ub

def pick_cpoints(X_all, y_all, N_u):
    idx = np.random.choice(np.shape(X_all)[0], N_u, replace=False)
    X, y = X_all[idx,:], y_all[idx,:]
    return X, y

def add_noise(u_train, noise):
    noisy_u_train = u_train + noise*np.std(u_train)*np.random.randn(u_train.shape[0], u_train.shape[1])
    return noisy_u_train

In [ ]:
def main_loop(N_u, noise, num_layers, num_units, activation):
    X_all, y_all, lb, ub = get_all(data_fpath)
    X_train, y_train = pick_cpoints(X_all, y_all, N_u)
    if noise:
        y_train = add_noise(y_train, noise)

    model = PhysicsInformedNN(X_train, y_train, num_layers, num_units, lb, ub, activation)
    model.train(nIter_Adam=5000, use_LBFGSB=True)

    u_pred, f_pred = model.predict(X_all)
    lambda_1_val = model.lambda_1
    lambda_2_val = np.exp(model.lambda_2)
    
    error_u = np.divide(np.linalg.norm(y_all - u_pred, 2), np.linalg.norm(y_all, 2))
    pct_error_l1 = np.abs(lambda_1_val - lambda_1_true) * 100
    pct_error_l2 = (np.divide(np.abs(lambda_2_val - lambda_2_true), (0.01/np.pi))) * 100

    print('Error u: %e' % (error_u))
    print('Error l1: %.5f%%' % (pct_error_l1[0]))
    print('Error l2: %.5f%%' % (pct_error_l2[0]))

    return pct_error_l1[0], pct_error_l2[0]

In [ ]:
N_u = [500, 1000, 1500, 2000]
noise = [0.0, 0.01, 0.05, 0.1]
num_layers = [2, 4, 6, 8]
num_units = [10, 20, 40]
error_lambda_1_table_1 = np.zeros((len(N_u), len(noise)))
error_lambda_2_table_1 = np.zeros((len(N_u), len(noise)))
error_lambda_1_table_2 = np.zeros((len(num_layers), len(num_units)))
error_lambda_2_table_2 = np.zeros((len(num_layers), len(num_units)))

In [ ]:
start_time = time.time()
print('*'*50, 'Beginning systemic discovery loop', '*'*50)

for i in range(len(N_u)):
    for j in range(len(noise)):
        print('*'*30, 'Beginning discovery: ', 'Num. of collocation points:', N_u[i], ', Noise:', int(noise[j]*100), '%', '*'*30)
        error_lambda_1_table_1[i,j], error_lambda_2_table_1[i,j] = main_loop(N_u[i], noise[j], num_layers[-1], num_units[-1], 'relu')
        print('*'*50, 'Discovery complete', '*'*50)

for i in range(len(num_layers)):
    for j in range(len(num_units)):
        print('*'*30, 'Beginning discovery: ', 'Num. of surrogate layers:', num_layers[i], ', Num. of surrogate units:', num_units[j], '*'*30)
        error_lambda_1_table_2[i,j], error_lambda_2_table_2[i,j] = main_loop(N_u[-1], noise[0], num_layers[i], num_units[j], 'relu')
        print('*'*50, 'Discovery complete', '*'*50)

print('*'*50, 'Systemic discovery loop complete', '*'*50)
end_time = time.time()
print('*'*50, "Time to completion:", (np.abs(start_time - end_time)), '*'*50)

In [ ]:
np.savetxt('./tables/error_lambda_1_table_1.csv', error_lambda_1_table_1, delimiter=' & ', fmt='$%2.3f$', newline=' \\\\\n')
np.savetxt('./tables/error_lambda_2_table_1.csv', error_lambda_2_table_1, delimiter=' & ', fmt='$%2.3f$', newline=' \\\\\n')
np.savetxt('./tables/error_lambda_1_table_2.csv', error_lambda_1_table_2, delimiter=' & ', fmt='$%2.3f$', newline=' \\\\\n')
np.savetxt('./tables/error_lambda_2_table_2.csv', error_lambda_2_table_2, delimiter=' & ', fmt='$%2.3f$', newline=' \\\\\n')

In [28]:
k_table1 = [[0.085, 0.102, 1.128, 0.216, 11.195, 0.566, 2.456, 25.478], [0.029, 0.047, 0.464, 0.868, 0.704, 0.144, 1.788, 6.800], [0.059, 0.029, 0.635, 0.015, 1.103, 0.829, 2.183, 6.265], [0.006, 0.105, 0.890, 0.551, 0.173, 0.689, 4.487, 4.877]]

In [29]:
k_table2 = [[29.896, 18.328, 24.828, 107.581, 16.358, 34.267], [1.402, 0.239, 0.156, 12.127, 2.268, 3.718], [0.049, 0.112, 0.052, 0.710, 0.078, 0.042], [0.343, 0.025, 0.041, 1.772, 0.329, 0.598]]

In [30]:
raissi_table1 = [[0.131, 0.518, 0.118, 1.319, 13.885, 0.483, 1.708, 4.058], [0.186, 0.533, 0.157, 1.869, 3.719, 8.262, 3.481, 14.544],[0.432, 0.033, 0.706, 0.725, 3.093, 1.423, 0.502, 3.156], [0.096, 0.039, 0.190, 0.101, 0.469, 0.008, 6.216, 6.391]]

In [31]:
raissi_table2 = [[11.696, 2.837, 1.679, 103.919, 67.055, 49.186], [0.332, 0.109, 0.428, 4.721, 1.234, 6.170], [0.668, 0.629, 0.118, 3.144, 3.123, 1.158], [0.414, 0.141, 0.266, 8.459, 1.902, 1.552]]

In [35]:
diffs_t1 = []
for i in range(len(k_table1)):
    diff = []
    for j in range(len(k_table1[i])):
        d = raissi_table1[i][j] - k_table1[i][j]
        diff.append(d)
    diffs_t1.append(diff)

diffs_t2 = []
for i in range(len(k_table2)):
    diff = []
    for j in range(len(k_table2[i])):
        d = raissi_table2[i][j] - k_table2[i][j]
        diff.append(d)
    diffs_t2.append(diff)

In [38]:
diffs_t2

[[-18.200000000000003,
  -15.491,
  -23.149,
  -3.662000000000006,
  50.697,
  14.918999999999997],
 [-1.0699999999999998,
  -0.13,
  0.272,
  -7.406000000000001,
  -1.0339999999999998,
  2.452],
 [0.619, 0.517, 0.066, 2.434, 3.0450000000000004, 1.1159999999999999],
 [0.07099999999999995,
  0.11599999999999999,
  0.225,
  6.686999999999999,
  1.573,
  0.9540000000000001]]